## Data test train source split

In [1]:
# storing file names
import numpy as np 
import pandas as pd 

#src_class = input('BH/NS : ')
src_class = "NS"
frac = 1

data = pd.read_csv('source_list/'+src_class+'_data_clean_source_list.csv')
data =  data.sample(frac=1)
frac_no = int(len(data)*frac)
train_data = data.iloc[:frac_no]
test_data = data.iloc[frac_no:]
train_data.to_csv('source_list/final_list/'+src_class+'_train.csv')

import os
ctg = src_class

def hr():
    print('----------------------------------------------------------------------------')

# Copy sources data to respective folders

data = pd.read_csv('source_list/final_list/'+ctg+'_train.csv')
ch_names = [di.replace(' ' ,'_')[:-1] for di in data['B_NAME']]
a_names = [di.replace(' ' ,'_')[:-1]+'.csv' for di in data['A_NAME']]
src_id = [si[:6] for si in data['SRC_ID']]
#print(src_id)
os.system('rm -r '+ctg+'_data/train/')
os.system('mkdir '+ctg+'_data/train/')
for c,s ,a in zip(ch_names , src_id , a_names):
    try:
        src = ctg+'_data_all/'+c+'.csv' 
        ## adding source_id in file names
        dest = ctg+'_data/train/'+ s+'-'+c+'#'+a 
        #print('Copying' , c )
        os.system('cp '+src+' '+dest)
    except:
        hr()
        print('check source : ' , c )
        hr()

hr()
print('TRAINING DATA STORED')

----------------------------------------------------------------------------
TRAINING DATA STORED


## Append all data in one file

In [2]:
from ipywidgets import IntProgress
from IPython.display import display

src_class = src_class

def hr():
    print('_______________________________________________________________________________________')


def to_float(d):
    temp = []
    for di in d:
        
        try:
            temp.append(float(di))
        except Exception as e:
            print(e)
            print('NaN detected resetting value')
            temp.append(np.NAN)
    temp = np.asarray(temp)
    return temp 



In [3]:
set_type = 'train'
os.system('ls '+src_class+'_data/'+set_type+'/ > file_list')
file_list = pd.read_csv('file_list' , names=['names'])['names']
data = pd.DataFrame()

for file in file_list:
    data_temp =  pd.read_csv(src_class+'_data/'+set_type+'/'+file)
    ## extracting source_id from file names 
    src_id_name = file[:6]
    src_id = [src_id_name]*len(data_temp)
    src_id = np.asarray(src_id)
    data_temp.insert(0 ,'src_id' , src_id)
    src_n_name = file[29:-4].replace('_' , ' ')
    src_n = [src_n_name]*len(data_temp)
    src_n = np.asarray(src_n)
    data_temp.insert(0 ,'src_n' , src_n)
    data =  data.append(data_temp)
data = data.replace('      NaN' , np.nan)
data.to_csv('raw_data/'+src_class+'_'+set_type+'.csv')
data.describe()

hr()

_______________________________________________________________________________________


# Data Processing

## Filtering

In [4]:
flag_params = ['streak_src_flag' , 'sat_src_flag' , 'pileup_flag' , 'mstr_sat_src_flag' , ]


In [5]:
def correct_bool(arr):
    temp = []
    #print(arr)
    for a in arr:
        #print(a , type(a))
        if(type(a) == bool):
            temp.append(a)
        else:
            if(a==' TRUE' or a=='True' or a=='  NAN'):
                temp.append(True)
            elif(a=='FALSE' or a=='False'):
                temp.append(False)
            else:
                print('something wrong' , a , type(a))
    return temp

In [6]:
def to_float(d):
    temp = []
    for di in d:
        
        try:
            temp.append(float(di))
        except Exception as e:
            print(e)
            print('NaN detected resetting value')
            temp.append(np.NAN)
    temp = np.asarray(temp)
    return temp 

In [8]:
data = pd.read_csv('raw_data/'+src_class+'_'+set_type+'.csv')
print('Data  Before flag filtering')
display(data.describe()[:1])
for f in flag_params:
    temp_flag = data[f]
    temp_flag = correct_bool(temp_flag)
    data[f] = temp_flag
    #print(temp_flag)
    data = data[data[f]==False]

#display(data)
data = data.dropna(how='all' , axis=1)
data = data.dropna(how='all' , axis=0)
display(data.describe()[:1])

Data  Before flag filtering


,Unnamed: 0,index,sepn,ra,dec,obsid,obi,src_rate_aper_m,src_rate_aper_lolim_m,src_rate_aper_hilim_m,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
count,493.0,493.0,493.0,493.0,493.0,493.0,493.0,462.0,462.0,462.0,...,383.0,198.0,198.0,196.0,493.0,493.0,493.0,493.0,493.0,493.0


,Unnamed: 0,index,sepn,ra,dec,obsid,obi,src_rate_aper_m,src_rate_aper_lolim_m,src_rate_aper_hilim_m,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
count,440.0,440.0,440.0,440.0,440.0,440.0,440.0,414.0,414.0,414.0,...,342.0,162.0,162.0,160.0,440.0,440.0,440.0,440.0,440.0,440.0


take average of Band params

In [9]:
band_params = [ 'photflux_aper' ,'photflux_aper_lolim' , 'photflux_aper_hilim' , 'flux_aper' , 'flux_aper_lolim' , 'flux_aper_hilim' , 'var_index' ,'var_prob' , 'ks_prob' , 'kp_prob' ,'var_sigma' ,'var_mean' , 'var_min' ,  'var_max' , 'var_inter_index' , 'var_inter_prob' , 'var_inter_sigma']

filters = ['_b' ,'_h' ,'_m' ,'_u' ,'_s']

combining band parameterd

In [10]:
data_band_mean = pd.DataFrame()
data_band_mean.index.name = 'name'
for param in band_params[:]:
    temp_row = [param+f for f in filters]
    data_temp = data[temp_row][:]

    data_temp_new = pd.DataFrame()
    for j in data_temp:
        data_temp_new.insert( 0 ,j ,to_float(data_temp[j].to_numpy()))
    temp_mean =  data_temp_new.mean(axis=1)
    data_band_mean.insert(0 , param , temp_mean)

display(data_band_mean)
#data_band_mean.to_csv('temp.csv')

,var_inter_sigma,var_inter_prob,var_inter_index,var_max,var_min,var_mean,var_sigma,kp_prob,ks_prob,var_prob,var_index,flux_aper_hilim,flux_aper_lolim,flux_aper,photflux_aper_hilim,photflux_aper_lolim,photflux_aper
name,,,,,,,,,,,,,,,,,
0,2.257511e-03,0.75960,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.053775e-15,0.000000e+00,0.000000e+00,5.583500e-07,0.000000e+00,0.000000e+00
1,2.257511e-03,0.75960,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.399667e-15,0.000000e+00,0.000000e+00,8.512333e-07,0.000000e+00,0.000000e+00
2,2.257511e-03,0.75960,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.485000e-16,0.000000e+00,0.000000e+00,7.035700e-07,0.000000e+00,2.738000e-08
3,2.257511e-03,0.75960,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.530075e-15,0.000000e+00,0.000000e+00,3.217000e-07,0.000000e+00,0.000000e+00
4,2.257511e-03,0.75960,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.696500e-15,0.000000e+00,0.000000e+00,4.229750e-07,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,3.685725e-07,0.65575,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.886780e-07,0.000000e+00,0.000000e+00
436,3.685725e-07,0.65575,4.0,0.000577,0.000430,0.000437,0.000022,0.200000,0.061000,0.357000,0.0,1.481875e-15,2.803750e-16,8.670500e-16,5.142000e-07,1.386400e-07,2.727180e-07
437,3.685725e-07,0.65575,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.548560e-06,0.000000e+00,0.000000e+00


Rescaling flux parameters

In [11]:
flux_params = ['photflux_aper' ,'photflux_aper_lolim' , 'photflux_aper_hilim' , 'flux_aper' , 'flux_aper_lolim' , 'flux_aper_hilim' ]
flux_params_band = []
for fp in flux_params:
    for f in filters:
        temp = fp+f
        flux_params_band.append(temp)
flux_params_extend = flux_params_band+flux_params

In [12]:
data_band_flux = data[flux_params_band]
data_band_flux = data_band_flux.reset_index(drop=True)
data_band_flux = data_band_flux.replace(0.0 , np.NAN)

display((data_band_flux))
data_mean_flux = data_band_mean[flux_params].reset_index(drop=True)
display(data_mean_flux)
data_flux = pd.concat([data_mean_flux , data_band_flux] , axis=1)
data_flux = data_flux.replace(0 , np.nan)
data_flux = data_flux.replace('0' , np.nan)
data_flux = data_flux.replace('0.0' , np.nan)
display(data_flux)
data_flux.to_csv('temp.csv')

,photflux_aper_b,photflux_aper_h,photflux_aper_m,photflux_aper_u,photflux_aper_s,photflux_aper_lolim_b,photflux_aper_lolim_h,photflux_aper_lolim_m,photflux_aper_lolim_u,photflux_aper_lolim_s,...,flux_aper_lolim_b,flux_aper_lolim_h,flux_aper_lolim_m,flux_aper_lolim_u,flux_aper_lolim_s,flux_aper_hilim_b,flux_aper_hilim_h,flux_aper_hilim_m,flux_aper_hilim_u,flux_aper_hilim_s
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.450000e-15,1.336000e-15,8.065000e-16,NaN,6.226000e-16
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.022000e-15,3.030000e-15,1.147000e-15,NaN,NaN
2,NaN,1.369000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.059000e-15,1.046000e-15,2.098000e-16,1.765000e-15,1.627000e-16
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.745000e-15,2.273000e-15,5.694000e-16,NaN,5.329000e-16
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.899000e-15,6.753000e-15,5.767000e-16,NaN,5.573000e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
436,7.456000e-07,3.415000e-07,2.113000e-07,NaN,6.519000e-08,4.474000e-07,1.452000e-07,1.006000e-07,NaN,NaN,...,7.239000e-16,1.423000e-16,2.553000e-16,NaN,NaN,2.725000e-15,2.229000e-15,8.170000e-16,NaN,1.565000e-16
437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
438,NaN,1.538000e-06,NaN,NaN,NaN,NaN,7.469000e-07,NaN,NaN,NaN,...,NaN,3.776000e-15,NaN,NaN,NaN,NaN,1.360000e-14,NaN,NaN,NaN


,photflux_aper,photflux_aper_lolim,photflux_aper_hilim,flux_aper,flux_aper_lolim,flux_aper_hilim
0,0.000000e+00,0.000000e+00,5.583500e-07,0.000000e+00,0.000000e+00,1.053775e-15
1,0.000000e+00,0.000000e+00,8.512333e-07,0.000000e+00,0.000000e+00,2.399667e-15
2,2.738000e-08,0.000000e+00,7.035700e-07,0.000000e+00,0.000000e+00,8.485000e-16
3,0.000000e+00,0.000000e+00,3.217000e-07,0.000000e+00,0.000000e+00,1.530075e-15
4,0.000000e+00,0.000000e+00,4.229750e-07,0.000000e+00,0.000000e+00,3.696500e-15
...,...,...,...,...,...,...
435,0.000000e+00,0.000000e+00,1.886780e-07,NaN,NaN,NaN
436,2.727180e-07,1.386400e-07,5.142000e-07,8.670500e-16,2.803750e-16,1.481875e-15
437,0.000000e+00,0.000000e+00,1.548560e-06,NaN,NaN,NaN
438,1.538000e-06,7.469000e-07,2.329000e-06,8.812000e-15,3.776000e-15,1.360000e-14


,photflux_aper,photflux_aper_lolim,photflux_aper_hilim,flux_aper,flux_aper_lolim,flux_aper_hilim,photflux_aper_b,photflux_aper_h,photflux_aper_m,photflux_aper_u,...,flux_aper_lolim_b,flux_aper_lolim_h,flux_aper_lolim_m,flux_aper_lolim_u,flux_aper_lolim_s,flux_aper_hilim_b,flux_aper_hilim_h,flux_aper_hilim_m,flux_aper_hilim_u,flux_aper_hilim_s
0,NaN,NaN,5.583500e-07,NaN,NaN,1.053775e-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.450000e-15,1.336000e-15,8.065000e-16,NaN,6.226000e-16
1,NaN,NaN,8.512333e-07,NaN,NaN,2.399667e-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.022000e-15,3.030000e-15,1.147000e-15,NaN,NaN
2,2.738000e-08,NaN,7.035700e-07,NaN,NaN,8.485000e-16,NaN,1.369000e-07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.059000e-15,1.046000e-15,2.098000e-16,1.765000e-15,1.627000e-16
3,NaN,NaN,3.217000e-07,NaN,NaN,1.530075e-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.745000e-15,2.273000e-15,5.694000e-16,NaN,5.329000e-16
4,NaN,NaN,4.229750e-07,NaN,NaN,3.696500e-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.899000e-15,6.753000e-15,5.767000e-16,NaN,5.573000e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,NaN,NaN,1.886780e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
436,2.727180e-07,1.386400e-07,5.142000e-07,8.670500e-16,2.803750e-16,1.481875e-15,7.456000e-07,3.415000e-07,2.113000e-07,NaN,...,7.239000e-16,1.423000e-16,2.553000e-16,NaN,NaN,2.725000e-15,2.229000e-15,8.170000e-16,NaN,1.565000e-16
437,NaN,NaN,1.548560e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
438,1.538000e-06,7.469000e-07,2.329000e-06,8.812000e-15,3.776000e-15,1.360000e-14,NaN,1.538000e-06,NaN,NaN,...,NaN,3.776000e-15,NaN,NaN,NaN,NaN,1.360000e-14,NaN,NaN,NaN


In [13]:
data_flux_log = pd.DataFrame()
for f in data_flux:
    temp = data_flux[f]
    temp = np.log10(temp)
#    display(temp)
    data_flux_log.insert(0 , f , temp)

display(data_flux_log)

,flux_aper_hilim_s,flux_aper_hilim_u,flux_aper_hilim_m,flux_aper_hilim_h,flux_aper_hilim_b,flux_aper_lolim_s,flux_aper_lolim_u,flux_aper_lolim_m,flux_aper_lolim_h,flux_aper_lolim_b,...,photflux_aper_u,photflux_aper_m,photflux_aper_h,photflux_aper_b,flux_aper_hilim,flux_aper_lolim,flux_aper,photflux_aper_hilim,photflux_aper_lolim,photflux_aper
0,-15.205791,NaN,-15.093396,-14.874194,-14.838632,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-14.977252,NaN,NaN,-6.253093,NaN,NaN
1,NaN,NaN,-14.940437,-14.518557,-14.519706,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-14.619849,NaN,NaN,-6.069951,NaN,NaN
2,-15.788612,-14.753255,-15.678195,-14.980468,-14.975104,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-6.863597,NaN,-15.071348,NaN,NaN,-6.152693,NaN,-7.562567
3,-15.273354,NaN,-15.244583,-14.643401,-14.561458,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-14.815287,NaN,NaN,-6.492549,NaN,NaN
4,-15.253911,NaN,-15.239050,-14.170503,-14.161214,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-14.432209,NaN,NaN,-6.373685,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.724279,NaN,NaN
436,-15.805486,NaN,-15.087778,-14.651890,-14.564633,NaN,NaN,-15.592949,-15.846795,-15.140321,...,NaN,-6.675101,-6.466609,-6.127494,-14.829188,-15.552261,-15.061956,-6.288868,-6.858111,-6.564286
437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.810072,NaN,NaN
438,NaN,NaN,NaN,-13.866461,NaN,NaN,NaN,NaN,-14.422968,NaN,...,NaN,NaN,-5.813044,NaN,-13.866461,-14.422968,-14.054926,-5.632831,-6.126738,-5.813044


In [14]:
var_params = [ 'var_index' ,'var_prob' , 'ks_prob' , 'kp_prob' ,'var_sigma' ,'var_mean' , 'var_min' ,  'var_max' , 'var_inter_index' , 'var_inter_prob' , 'var_inter_sigma']
data_var = data_band_mean[var_params].reset_index(drop=True)
display(data_var)

,var_index,var_prob,ks_prob,kp_prob,var_sigma,var_mean,var_min,var_max,var_inter_index,var_inter_prob,var_inter_sigma
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2,0.75960,2.257511e-03
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2,0.75960,2.257511e-03
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2,0.75960,2.257511e-03
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2,0.75960,2.257511e-03
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2,0.75960,2.257511e-03
...,...,...,...,...,...,...,...,...,...,...,...
435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.65575,3.685725e-07
436,0.0,0.357000,0.061000,0.200000,0.000022,0.000437,0.000430,0.000577,4.0,0.65575,3.685725e-07
437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.65575,3.685725e-07
438,0.0,0.454000,0.610000,0.268000,0.000042,0.000561,0.000541,0.000779,0.0,0.33600,4.505500e-07


In [39]:
single_params = [ 'flux_powlaw'  , 'powlaw_gamma' , 'powlaw_nh' ,  'powlaw_ampl' ,  'powlaw_stat' , 'flux_bb' , 'flux_bb_hilim' , 'flux_bb_lolim' , 'bb_kt' , 'bb_nh' ,  'bb_ampl' ,  'bb_stat' , 'flux_brems' , 'brems_kt' , 'brems_nh' ,'brems_stat' ,]

hard_params = ['hard_hm' ,'hard_hm_hilim' ,'hard_hm_lolim' ,  'hard_hs' , 'hard_ms' ,'hard_hs_hilim' ,'hard_hs_lolim' ,'hard_ms_hilim' ,'hard_ms_lolim',]

model_fit_params = ['flux_powlaw'  , 'flux_powlaw_hilim' ,'flux_powlaw_lolim', 'powlaw_gamma' , 'powlaw_gamma_hilim' , 'powlaw_gamma_lolim' ,'powlaw_nh' , 'powlaw_nh_hilim' , 'powlaw_nh_lolim' ,  'powlaw_ampl' ,  'powlaw_ampl_hilim' , 'powlaw_ampl_lolim' , 'powlaw_stat' , 'flux_bb' , 'flux_bb_hilim' , 'flux_bb_lolim' , 'bb_kt' ,'bb_kt_hilim' , 'bb_kt_lolim' , 'bb_nh_hilim' , 'bb_nh_lolim' , 'bb_nh' ,  'bb_ampl' ,'bb_ampl_lolim' ,'bb_ampl_hilim' ,   'bb_stat' , 'flux_brems' , 'flux_brems_lolim' , 'flux_brems_hilim' , 'brems_kt' , 'brems_kt_hilim' ,'brems_kt_lolim' , 'brems_nh' , 'brems_nh_hilim' ,'brems_nh_lolim' ,'brems_stat' ,]

obs_info_params = [ 'livetime','significance','likelihood','pileup_flag','mstr_sat_src_flag','mstr_streak_src_flag'   ,'gti_obs' , 'flux_significance_b'  , 'flux_significance_m' , 'flux_significance_s' , 'flux_significance_h' , 'flux_significance_u'    ]

single_params_lim = [ ]

In [40]:
data_rest = data[hard_params+model_fit_params].reset_index(drop=True)
display(data_rest)


,hard_hm,hard_hm_hilim,hard_hm_lolim,hard_hs,hard_ms,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,flux_powlaw,...,flux_brems,flux_brems_lolim,flux_brems_hilim,brems_kt,brems_kt_hilim,brems_kt_lolim,brems_nh,brems_nh_hilim,brems_nh_lolim,brems_stat
0,0.9994,1.0000,0.0806,0.9994,-0.9994,1.0000,-0.1443,0.1755,-1.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.9994,1.0000,0.0244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.9994,1.0000,0.4116,0.9994,-0.9994,1.0000,0.1805,0.1493,-1.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.9994,1.0000,-0.2492,-0.9994,-0.9994,0.2017,-1.0000,0.0581,-1.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.9994,1.0000,0.0406,0.9994,-0.9994,1.0000,-0.2929,0.0344,-1.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,0.9994,1.0000,-0.1968,0.9994,-0.9994,1.0000,-1.0000,1.0000,-1.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
436,0.2936,0.6015,-0.0831,0.7146,0.5147,0.9413,0.3779,0.8314,0.1168,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
437,0.9981,1.0000,-0.2080,0.9994,-0.9994,1.0000,-0.2904,1.0000,-1.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
data_obs_info = data[obs_info_params].reset_index(drop=True)
display(data_obs_info)

,livetime,significance,likelihood,pileup_flag,mstr_sat_src_flag,mstr_streak_src_flag,gti_obs,flux_significance_b,flux_significance_m,flux_significance_s,flux_significance_h,flux_significance_u
0,35796.9,167.36,21.89294,False,False,False,2000-10-26T22:18:33,0.00,0.00,0.0,0.00,NaN
1,13506.7,167.36,21.89294,False,False,False,2001-07-14T02:05:23,0.00,0.00,NaN,0.00,NaN
2,37677.7,167.36,21.89294,False,False,False,2002-05-22T23:20:43,0.00,0.00,0.0,1.03,0.0
3,12369.5,167.36,21.89294,False,False,False,2002-02-19T14:43:23,0.00,0.00,0.0,0.00,NaN
4,11855.5,167.36,21.89294,False,False,False,2002-03-23T12:42:56,0.00,0.00,0.0,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
435,24046.1,2.50,12.57282,False,False,False,2003-06-27T09:09:58,0.00,0.00,0.0,0.00,0.0
436,49114.6,2.50,12.57282,False,False,False,2009-07-28T15:16:17,2.50,1.91,1.0,1.78,0.0
437,4909.3,2.50,12.57282,False,False,False,2009-08-10T08:26:55,0.00,0.00,0.0,0.00,0.0
438,10619.0,5.50,94.26352,False,False,False,2001-07-18T17:47:51,NaN,NaN,NaN,1.94,NaN


In [42]:
data_identify = data[['src_n' , 'src_id']].reset_index(drop=True)
print(data_identify)

                    src_n  src_id
0    1A 1742-289           NS0001
1    1A 1742-289           NS0001
2    1A 1742-289           NS0001
3    1A 1742-289           NS0001
4    1A 1742-289           NS0001
..                    ...     ...
435  4U1745-203            NS0080
436  4U1745-203            NS0080
437  4U1745-203            NS0080
438  GRO J1744-28          NS0084
439  GRO J1744-28          NS0084

[440 rows x 2 columns]


In [43]:
data_combined = pd.concat([data_identify , data_obs_info , data_flux_log , data_var , data_rest] , axis=1)
data_combined.insert( 0 , 'class' , [src_class]*len(data_combined))
data_combined = data_combined.reset_index(drop=True)
data_combined.index.name = 'index'
display(data_combined)
data_combined.to_csv('processed_data/'+src_class+'_.csv')

,class,src_n,src_id,livetime,significance,likelihood,pileup_flag,mstr_sat_src_flag,mstr_streak_src_flag,gti_obs,...,flux_brems,flux_brems_lolim,flux_brems_hilim,brems_kt,brems_kt_hilim,brems_kt_lolim,brems_nh,brems_nh_hilim,brems_nh_lolim,brems_stat
index,,,,,,,,,,,,,,,,,,,,,
0,NS,1A 1742-289,NS0001,35796.9,167.36,21.89294,False,False,False,2000-10-26T22:18:33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NS,1A 1742-289,NS0001,13506.7,167.36,21.89294,False,False,False,2001-07-14T02:05:23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NS,1A 1742-289,NS0001,37677.7,167.36,21.89294,False,False,False,2002-05-22T23:20:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NS,1A 1742-289,NS0001,12369.5,167.36,21.89294,False,False,False,2002-02-19T14:43:23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NS,1A 1742-289,NS0001,11855.5,167.36,21.89294,False,False,False,2002-03-23T12:42:56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,NS,4U1745-203,NS0080,24046.1,2.50,12.57282,False,False,False,2003-06-27T09:09:58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
436,NS,4U1745-203,NS0080,49114.6,2.50,12.57282,False,False,False,2009-07-28T15:16:17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
437,NS,4U1745-203,NS0080,4909.3,2.50,12.57282,False,False,False,2009-08-10T08:26:55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
